# Energy minimization of a closed shell under constant volume

In [1]:
#First we need to import the module for the simulations
import pymembrane as mb
#numpy
import numpy as np

#### Loading Mesh
We first load the mesh  with 6281 vertices and 12558 faces.

In [2]:
#create a system 
system = mb.System()
#read the mesh
vertex_file = 'vertices_R1.0.inp'
face_file = 'faces_R1.0.inp'
system.read_mesh_from_files(files = {'vertices': vertex_file, 'faces': face_file})

Mesh
 Numvertices  6281
 NumFaces  12558
 NumEdges  18837
 NumHEdges  37674


#### Create a dumper

In [3]:
dump = system.dump() 
dump.vtk("initial mesh")

#### Check the average edge length

In [4]:
compute = system.compute_mesh()
mean_edge_length = np.mean(compute.edge_lengths())
print('mean edge length: {}'.format(mean_edge_length))
volume = compute.volume()
print('volume: {}'.format(volume))

mean edge length: 0.04816973986825737
volume: 4.185092296647554


#### Create an Evolver to add the potentials

In [5]:
evolver = mb.Evolver(system)

# add some strain into the edges
k = 50.0
l0 = 1.1*mean_edge_length

evolver.add_force("Mesh>Harmonic", {"k": {"0": str(k)},
                                    "l0": {"0": str(l0)}})

# add bending rigidity
kb = 1.0
evolver.add_force("Mesh>Bending", {"kappa": {"0": str(kb)}})

In [6]:
energies = compute.energy(evolver)
print("initial total energy:", energies["vertices"] + energies["edges"] + energies["faces"])

initial total energy: 24.727914970797148


#### Add a minimizer

In [7]:
evolver.add_minimizer("Mesh>Fire", {"dt": str(1e-2), "max_iter": str(10000), "ftol": str(1e-2), "etol": str(1e-5)})

#### Add volume contraint

In [8]:
evolver.add_constraint("Mesh>Volume", {"V": str(volume), "max_iter": str(10000), "tol":str(1e-5)})

parameter  V  is not part of  volume  constraint
parameter  max_iter  is not part of  volume  constraint
parameter  V  is not part of  volume  constraint
parameter  max_iter  is not part of  volume  constraint


#### Minimize 

In [9]:
evolver.minimize()

In [10]:
print(evolver.get_minimizer_info())

{'Mesh>Fire': {'alpha': '6.47389e-09', 'dE': '-6.83481e-07', 'dt': '6.10352e-07', 'energy': '66.1324', 'etol': '1e-05', 'fnorm': '33.2405', 'ftol': '0.01', 'is_satisfy': 'false', 'n_since_start': '10000', 'name': 'fire'}}


In [11]:
energies = compute.energy(evolver)
print("final total energy:", energies["vertices"] + energies["edges"] + energies["faces"])

new_volume = compute.volume()
print("volume difference: {}".format(volume-new_volume))

final total energy: 66.1323794478285
volume difference: -1.2166669458579804e-06


In [12]:
dump.vtk("new mesh")